# Linear Model [Logistic Regression]

### Import dependencies

In [1]:
import os
import sys
import pickle
from datetime import datetime as dt

import tensorflow as tf
import numpy as np

from tqdm import tqdm

import matplotlib.pyplot as plt
from matplotlib import style
style.use('ggplot')

%matplotlib inline

### Load in the data

In [2]:
from tensorflow.examples.tutorials.mnist import input_data


save_dir = '../saved/logistic-regression'
data_dir = '../datasets/MNIST'
saved_data = os.path.join(save_dir, f'data/{os.path.basename(data_dir)}.pkl')

if not os.path.isfile(saved_data):
    start = dt.now()
    data = input_data.read_data_sets('../datasets/MNIST/', one_hot=True)
    print(f'Took {dt.now() - start}')
    if not os.path.exists(os.path.dirname(saved_data)):
        os.makedirs(os.path.dirname(saved_data))
    pickle.dump(file=open(saved_data, 'wb'), obj=data)
    
    print('\nCached data for future use.')
else:
    start = dt.now()
    data = pickle.load(file=open(saved_data, 'rb'))
    print('Loaded cached data.')
    print(f'Took {dt.now() - start}')
del start

Loaded cached data.
Took 0:00:00.297999


In [3]:
print('Training data   = {:,}'.format(len(data.train.labels)))
print('Testing data    = {:,}'.format(len(data.test.labels)))
print('Validation data =  {:,}'.format(len(data.validation.labels)))

Training data   = 55,000
Testing data    = 10,000
Validation data =  5,000


### Hyperparameters

In [4]:
# Input
image_size = 28
image_shape = (image_size, image_size)
image_size_flat = image_size * image_size
num_classes = 10

# Training
save_step = 500
num_iter = 0
val_batch = 50
test_batch = 70
train_batch = 200
learning_rate = 1e-3

## Building the _Computational Graph_

### Placeholder variables

In [5]:
X = tf.placeholder(tf.float32, [None, image_size_flat])
y = tf.placeholder(tf.float32, [None, num_classes])
y_cls = tf.argmax(y, axis=1)

### Trainable variables

In [6]:
W = tf.Variable(tf.truncated_normal(shape=[image_size_flat, num_classes]))
b = tf.Variable(tf.zeros(shape=[num_classes]))

### Forward Propagation

In [7]:
logits = tf.add(tf.matmul(X, W), b)
y_pred = tf.nn.softmax(logits)
y_pred_cls = tf.argmax(y_pred, axis=1)

### Cost function and Gradient Descent

In [8]:
xentropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y)
loss = tf.reduce_mean(xentropy)
# Optimizer
global_step = tf.Variable(0, trainable=False)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_step = optimizer.minimize(loss, global_step=global_step)

### Performance measure

In [9]:
correct = tf.equal(y_pred_cls, y_cls)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

## Running the Computational Graph

### Define `tf.Session`

In [10]:
sess = tf.Session()

#### _initializing model/global variables_

In [11]:
def init():
    global num_iter
    __init = tf.global_variables_initializer()
    sess.run(__init)
    num_iter = 0
init()  # Initialize to start with

In [12]:
tensorboard_dir = os.path.join(save_dir, 'tensorboard')
logdir = os.path.join(tensorboard_dir, 'log')

model_dir = os.path.join(save_dir, 'models')
model_path = os.path.join(model_dir, 'model.ckpt')

# Histograms
tf.summary.histogram('W', values=W, family='params')
tf.summary.histogram('b', values=b, family='params')
# Scalars
tf.summary.scalar('loss', tensor=loss, family='evaluation')
tf.summary.scalar('accuracy', tensor=accuracy, family='evaluation')
# merged...
merged = tf.summary.merge_all()

# Savers and writers
saver = tf.train.Saver()
writer = tf.summary.FileWriter(logdir=logdir, graph = sess.graph)

### Maybe restore checkpoint

In [13]:
if tf.gfile.Exists(model_dir):
    try:
        print('INFO: Attempting to restore last checkpoint')
        last_ckpt = tf.train.latest_checkpoint(model_dir)
        saver.restore(sess=sess, save_path=last_ckpt)
        print(f'INFO: Successfully restored last checkpoint {last_ckpt}')
    except Exception as e:
        sys.stderr.write(f'ERR: Could not restore checkpoint. {e}')
        sys.stderr.flush()
else:
    tf.gfile.MakeDirs(model_dir)
    print(f'INFO: Created checkpoint dir @ {model_dir}')

INFO: Attempting to restore last checkpoint
INFO:tensorflow:Restoring parameters from ../saved/logistic-regression/models/model.ckpt-10903
INFO: Successfully restored last checkpoint ../saved/logistic-regression/models/model.ckpt-10903


### Define Some Helper functions

#### _Train the model_

In [14]:
def train(iterations=1000):
    global num_iter
    start = dt.now()
    for _ in range(iterations):
        # Increment the iteration counter.
        num_iter += 1
        # Get batches
        X_batch, y_batch = data.train.next_batch(train_batch)
        feed_dict = {X: X_batch, y: y_batch}
        _, i_global = sess.run([train_step, global_step], feed_dict=feed_dict)
        if num_iter % save_step == 0:
            saver.save(sess=sess, save_path=model_path, global_step=global_step)
            summary = sess.run(merged, feed_dict=feed_dict)
            writer.add_summary(summary, global_step=i_global)
        sys.stdout.write(f'\rIter: {num_iter:,}\tGlobal iter: {i_global:,}'
                         f'\tTime taken: {dt.now() - start}')
    print()
    print(f"{80*'='}")
    print(f'\tCompleted {num_iter:,} iterations.')
    print(f"{80*'='}")

#### Run accuracy

In [15]:
def score(test=True, validation=False, use_batch=True):
    print(80*'=')
    print('Accuracy after {:,} iterations'.format(num_iter))
    feed_dict = None
    if test:
        if use_batch:
            X_batch, y_batch = data.test.next_batch(test_batch)
            feed_dict = {X: X_batch, y: y_batch}
        else:
            feed_dict = {X: data.test.images, y: data.test.labels}
        acc = sess.run(accuracy, feed_dict=feed_dict)
        print('Accuracy on test set: {:.02%}'.format(acc))
    if validation:
        if use_batch:
            X_batch, y_batch = data.validation.next_batch(val_batch)
            feed_dict = {X: X_batch, y: y_batch}
        else:
            feed_dict = {X: data.validation.images, y: data.validation.labels}
        acc = sess.run(accuracy, feed_dict=feed_dict)
        print('Accuracy on validation set: {:.02%}'.format(acc))
    print(80*'=')

## Training the model

In [16]:
train(iterations=10)
score(test=True, use_batch=False)

Iter: 10	Global iter: 10,913	Time taken: 0:00:00.165819
	Completed 10 iterations.
Accuracy after 10 iterations
Accuracy on test set: 91.43%


In [17]:
train(iterations=90)
score(test=True, use_batch=False)

Iter: 100	Global iter: 11,003	Time taken: 0:00:00.426163
	Completed 100 iterations.
Accuracy after 100 iterations
Accuracy on test set: 91.39%


In [18]:
train(iterations=900)
score(test=True, use_batch=True)

Iter: 1,000	Global iter: 11,903	Time taken: 0:00:03.770662
	Completed 1,000 iterations.
Accuracy after 1,000 iterations
Accuracy on test set: 95.71%


In [19]:
train(iterations=9000)
score(test=True, validation=True, use_batch=True)

Iter: 10,000	Global iter: 20,903	Time taken: 0:00:34.495049
	Completed 10,000 iterations.
Accuracy after 10,000 iterations
Accuracy on test set: 94.29%
Accuracy on validation set: 94.00%


### Clear cached files

In [ ]:
import shutil

# Clear saved mnist `data`
shutil.rmtree(os.path.dirname(saved_data))

In [ ]:
# sess.close()